In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
BATCH_SIZE = 2
IMG_H, IMG_W = 224, 224
DATA_DIR = "./food101/images/"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_H, IMG_W),
  batch_size=BATCH_SIZE,
  label_mode="categorical")

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_H, IMG_W),
  batch_size=BATCH_SIZE,
  label_mode="categorical")

In [ ]:
class_names = train_ds.class_names
class_names

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
INPUT_SHAPE = (IMG_H, IMG_W, 3)
NUM_OUTPUTS = len(train_ds.class_names)

In [ ]:
INPUT_SHAPE

In [ ]:
NUM_OUTPUTS

In [ ]:
model = Sequential()

# conv1
model.add(Conv2D(filters=64, input_shape=INPUT_SHAPE, kernel_size=(7,7), strides=2, activation="relu"))

# conv2
model.add(MaxPooling2D(pool_size=(3,3), strides=2))

for i in range(3):
    model.add(Conv2D(filters=128, kernel_size=(1,1), activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", groups=32))
    model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))

# conv3
for i in range(4):
    model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu", groups=32))
    model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))

# conv4
for i in range(6):
    model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu", groups=32))
    model.add(Conv2D(filters=1024, kernel_size=(1,1), activation="relu"))

# conv5
for i in range(3):
    model.add(Conv2D(filters=1024, kernel_size=(1,1), activation="relu"))
    model.add(Conv2D(filters=1024, kernel_size=(3,3), activation="relu", groups=32))
    model.add(Conv2D(filters=2048, kernel_size=(1,1), activation="relu"))

# final layer
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(NUM_OUTPUTS, activation="softmax"))

# optimizer
adam = Adam(learning_rate=0.01)
# model checkpoint
checkpoint_filePath = "./tmp/checkpoint/"
checkpoint = ModelCheckpoint(checkpoint_filePath, monitor="categorical_accuracy", verbose=1, save_best_only=True)
# compiling the model
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["categorical_accuracy"])
model.summary()

In [ ]:
EPOCHS = 2
history =   model.fit(train_ds,
            epochs=EPOCHS,
            callbacks=[checkpoint],
            validation_data=val_ds)